## Ejercicio: Análisis de una Competencia de Kaggle

### Contexto:

Tienes un conjunto de datos en formato JSON que contiene información sobre una competencia de Kaggle. Los datos incluyen el ID del participante, el número de intentos, el tiempo invertido en la competencia y las puntuaciones obtenidas en cada intento. Algunos participantes tienen datos incompletos o valores nulos, que deberás manejar antes de realizar el análisis.

### Objetivos:

1.	Cargar y explorar un archivo JSON.
2.	Identificar y manejar datos nulos.
3.	Calcular estadísticas descriptivas (promedios, máximos, mínimos, etc.) para el número de intentos, el tiempo invertido y las puntuaciones.
4.	Aplicar condicionales y operadores matemáticos para evaluar el rendimiento de los participantes (e.g., calcular el promedio de puntuaciones por intento).
5.	Guardar el conjunto de datos limpio en un nuevo archivo JSON.

In [5]:
# Importar Dependencias y recuperar el Archivo
import numpy as np
import pandas as pd
import json

path_json = '../data/ejercicios_integracion/competencia_kaggle.json'
df = pd.read_json(path_json)
print(df.shape)
df.head(5)

(5, 5)


,id_participante,nombre,intentos,tiempo_total_horas,puntuaciones
0,1,Participante_A,5,12.0,"[80, 85, 90, 88, None]"
1,2,Participante_B,3,NaN,"[70, 75, None]"
2,3,Participante_C,4,10.0,"[85, 90, 95, 93]"
3,4,Participante_D,2,5.0,"[60, None]"
4,5,Participante_E,6,14.0,"[78, 82, None, None, 80, 85]"


In [6]:
# Reemplazar los valores NaN o nullos con 0
df.fillna(0, inplace=True)
df.head(5)

,id_participante,nombre,intentos,tiempo_total_horas,puntuaciones
0,1,Participante_A,5,12.0,"[80, 85, 90, 88, None]"
1,2,Participante_B,3,0.0,"[70, 75, None]"
2,3,Participante_C,4,10.0,"[85, 90, 95, 93]"
3,4,Participante_D,2,5.0,"[60, None]"
4,5,Participante_E,6,14.0,"[78, 82, None, None, 80, 85]"


In [12]:
# Convertir las columnas a arrays de numpy
id_participantes = df['id_participante'].values
nombres = df['nombre'].values
intentos = df['intentos'].values
tiempo_total_horas = df['tiempo_total_horas'].values


In [14]:
# Identificar al participante con menos intentos
indice_menos_intentos = np.argmin(intentos)
participante_menos_intentos = nombres[indice_menos_intentos]

# Identificar al participante con más intentos
indice_mas_intentos = np.argmax(intentos)
participante_mas_intentos = nombres[indice_mas_intentos]

# Identificar al participante con más tiempo total de horas
indice_mas_tiempo = np.argmax(tiempo_total_horas)
participante_mas_tiempo = nombres[indice_mas_tiempo]

# Sumar los puntos logrados por cada participante
total_puntos_por_participante = []
for index, row in df.iterrows():
    puntuaciones = row['puntuaciones']
    total_puntos = sum([punto if punto is not None else 0 for punto in puntuaciones])
    total_puntos_por_participante.append({"participante": row['nombre'], "total_puntos": total_puntos})

# Imprimir los resultados
print(f"El participante con menos intentos es: {participante_menos_intentos} con {intentos[indice_menos_intentos]} intentos.")
print(f"El participante con más intentos es: {participante_mas_intentos} con {intentos[indice_mas_intentos]} intentos.")
print(f"El participante con más tiempo total en horas es: {participante_mas_tiempo} con {tiempo_total_horas[indice_mas_tiempo]} horas.")
print("Resultado total de puntos por participante:")
print(total_puntos_por_participante)

El participante con menos intentos es: Participante_D con 2 intentos.
El participante con más intentos es: Participante_E con 6 intentos.
El participante con más tiempo total en horas es: Participante_E con 14.0 horas.
Resultado total de puntos por participante: [{'participante': 'Participante_A', 'total_puntos': 343}, {'participante': 'Participante_B', 'total_puntos': 145}, {'participante': 'Participante_C', 'total_puntos': 363}, {'participante': 'Participante_D', 'total_puntos': 60}, {'participante': 'Participante_E', 'total_puntos': 325}]
